# Librerias

In [1]:
import importlib
import os, shutil
import numpy as np
import pandas as pd
import mplcursors as mpc

from tkinter import *
from tkinter.messagebox import askokcancel

from IPython.display import clear_output

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'browser'

import matplotlib.pyplot as plt   # plotting


import rai_exp_2023_sivarita as exp
importlib.reload(exp)

import Sivarita
importlib.reload(Sivarita)

<module 'Sivarita' from 'c:\\repositorios\\rai_exp_2023_sivarita\\Sivarita.py'>

# Preparar datos

Sincronizar los datos de la carpeta <font color='orange'>**/\_data\_**</font> con la carpeta <font color='orange'>**/Experimentaciones/2023_sivarita/\_data\_**</font> de teams.

<font color='red'>**[IMPORTANTE]**</font> Es necesario tener sincronizado el canal [Experimentaciones](https://miumh.sharepoint.com/:f:/s/NBIO/ErXH5_wtbyZBpVINdmT_GRUBNEXJmMrEtJXeEJYZERJYrA?e=WnxzMY) de Teams con el ordenador.

In [ ]:
if not "dataSyncCheck" in locals():
    exp.downloadDataFromTeams("/Resultados/_data_")

dataSyncCheck = True

Cargar excel con los usuarios

In [2]:
path = "./_data_/"

df_brazo = pd.read_excel(path + "participantes_datos_analisis.xlsx")
df_brazo.sample(9)

,nombre,ID,brazo,FM,grupo
42,Victoria_Payá,45,R,66,sano
12,Francisco_Ballester Marin,13,R,66,leve
9,Alberto_Villegas,10,L,66,leve
20,Fernando_Ruiz Santa Olalla,22,R,23,moderado
33,Jose Tomas_Torralbo,35,L,42,moderado
4,Isabel_Cuenca,5,R,66,leve
10,Teresa_García,11,R,64,leve
24,Francisco Javier_Fernandez Lerma,26,L,66,leve
25,Manuel_Torres Marti,27,L,65,leve


Cargar datos usuarios

In [3]:
#Path
path = "./_data_/Usuarios/"

#Cargar datos
df_data = exp.loadData(path)
df_data.sample(5)

,usuario,sesion,actividad,tipo_actividad,upper_size,fore_size,brazo,num_rep,dataFrame,dataFrameIA
326,Luis Daniel_Lledó,2023_10_10_11_46_33,TouchGame,HOMBRO,345.0,300.0,R,4.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
252,Jose Andres_Ferrer Ferrer,2023_5_12_11_22_55,PaintForms,TRIÁNGULO,350.0,260.0,L,4.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...
111,Carmen_Cerrillo,2023_5_5_12_0_46,PaintForms,TRIÁNGULO,290.0,250.0,R,4.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
267,Jose Tomas_Torralbo,2023_8_9_10_39_21,PaintForms,CUADRADO,360.0,280.0,L,3.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
45,Andrea_Blanco,2023_10_10_8_51_21,Move_Cube,SORTEAR,270.0,240.0,R,4.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...


Filtrar datos erróneos

In [4]:
count_vacio = 0
count_correcto = 0

#Filtrar DataFrames IA vacios
for i_row in range(df_data.shape[0]):
    if df_data.dataFrameIA[i_row].empty is True:
        print("Usuario: {0} -- Sesion: {1}".format(df_data.usuario[i_row], (df_data.sesion[i_row])))
        df_data = df_data.drop(i_row)
        count_vacio = count_vacio + 1
    else:
        count_correcto = count_correcto + 1

df_data.reset_index(drop=True, inplace=True)
print("Correctos: {0}".format(count_correcto))
print("Vacíos: {0}".format(count_vacio))

#Filtrar Dimensiones de brazo fuera de rango
for i_row in range(df_data.shape[0]):
    if (df_data.upper_size[i_row] + df_data.fore_size[i_row]) < 450 or (df_data.upper_size[i_row] + df_data.fore_size[i_row]) > 700:
        df_data = df_data.drop(i_row)

df_data.reset_index(drop=True, inplace=True)

df_data.sample(5)



Correctos: 466
Vacíos: 0


,usuario,sesion,actividad,tipo_actividad,upper_size,fore_size,brazo,num_rep,dataFrame,dataFrameIA
76,Antonio_Montes,2023_7_10_10_35_57,Move_Cube,SORTEAR,315.0,290.0,R,4.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...
8,Abigail_Jaen Llorens,2023_8_14_11_17_48,TouchGame,HOMBRO,310.0,270.0,R,3.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
316,Luis Daniel_Lledó,2023_10_10_11_47_49,TouchGame,CABEZA,345.0,300.0,R,4.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
3,Abigail_Jaen Llorens,2023_8_14_11_11_47,Move_Cube,SORTEAR,310.0,270.0,R,3.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...
7,Abigail_Jaen Llorens,2023_8_14_11_16_43,PaintForms,CÍRCULO,310.0,270.0,R,3.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...


# Procesar Datos

In [5]:
# Obtener los rangos articulares para cada paciente
df_params = exp.process_all_params(df_data)

In [6]:
#Filtrar movimientos articulares
condicion = (abs(df_params.iloc[:,-14:]) > 180).any(axis=1)
df_params = df_params[~condicion]
df_params.sample(5)

,usuario,session,activity,tipo_actividad,brazo,DTW_org_error,DTW_error,Porcentaje_pred,MAX_Q1,MAX_Q2,...,MAX_Q5,MAX_Q6,MAX_Q7,MIN_Q1,MIN_Q2,MIN_Q3,MIN_Q4,MIN_Q5,MIN_Q6,MIN_Q7
268,José Vicente_García,2023_10_10_13_29_14,PaintForms,CUADRADO,R,8.355548,5.199270,0.000000,6.489746,40.855545,...,31.334678,-3.605817,-13.997463,-30.404104,-1.969291,-77.202808,-45.129315,-8.943099,-15.517375,-34.606708
18,Alberto_Villegas,2023_3_27_13_0_19,TouchGame,RODILLA,R,1.920949,6.430297,79.166667,14.498555,8.450027,...,-0.612408,14.547704,-7.942774,-41.548101,-18.696630,-52.376132,-28.819017,-32.937911,2.240369,-33.470797
164,Esther_Blas Cruz,2023_6_12_11_38_8,MoveGlass,LLENADO,L,10.881067,7.126356,0.000000,14.885488,7.393901,...,97.184545,2.861314,13.169101,-9.742609,-3.849584,-27.599393,-8.114239,54.803454,-12.566092,-8.876962
430,Victoria_Payá,2023_10_10_9_24_58,Move_Cube,DESPLAZAR,R,2.316000,2.316000,100.000000,2.323250,28.024981,...,57.578581,6.395726,24.469446,-37.776036,-1.097327,-56.864568,-21.350249,-6.463224,-10.277720,-16.877301
376,Raúl_Martín,2023_10_10_12_38_59,MoveGlass,LLENADO,R,3.121223,3.121223,100.000000,15.763205,43.817748,...,87.422775,13.136510,53.503171,2.990150,1.149231,-27.064147,-29.983113,-11.007868,-9.216494,1.962619


In [7]:
#Filtrar por cantidad de ejercicios correctos
df_params = exp.filter_per_Activities(df_params)



In [ ]:
#Filtrar por cantidad de actividades
conteo_nombres = df_params['usuario'].value_counts()
print(conteo_nombres)
# Filtra los usuarios que se repiten más de una vez
usuarios_a_mantener = conteo_nombres[conteo_nombres > 1].index
print(len(usuarios_a_mantener))
# Crea un nuevo DataFrame con solo los usuarios que se repiten más de una vez
df_params = df_params[df_params['usuario'].isin(usuarios_a_mantener)]

df_params.sample(5)


Filtrar por Usuario

In [21]:
df_user = df_params[df_params['usuario'] == 'Filomena_Almansa Consuegra']
print(df_user)


                        usuario             session    activity  \
178  Filomena_Almansa Consuegra  2023_3_29_15_33_56   MoveGlass   
179  Filomena_Almansa Consuegra  2023_3_29_15_36_34   MoveGlass   
180  Filomena_Almansa Consuegra  2023_3_29_15_38_37   Move_Cube   
181  Filomena_Almansa Consuegra  2023_3_29_15_40_12   Move_Cube   
184  Filomena_Almansa Consuegra   2023_3_29_15_48_7  PaintForms   
185  Filomena_Almansa Consuegra  2023_3_29_15_51_48  PaintForms   
186  Filomena_Almansa Consuegra  2023_3_29_15_53_56   TouchGame   
187  Filomena_Almansa Consuegra  2023_3_29_15_56_13   TouchGame   
188  Filomena_Almansa Consuegra  2023_3_29_15_57_22   TouchGame   

    tipo_actividad brazo  DTW_org_error   DTW_error  Porcentaje_pred  \
178          BEBER     R      10.392828   10.392828       100.000000   
179        LLENADO     R       5.797528    5.797528       100.000000   
180      DEPOSITAR     R       7.278851    5.898852         0.000000   
181        SORTEAR     R       6.538979  

Filtrar por Grupos

In [8]:
# for itere grupos
hawer = df_brazo.grupo.unique()
df_group = []
group_list = []
for group in hawer:
    # filtrar grupo
    group_list.append(group)
    #df_group.append(exp.process_byGroups(df_brazo, df_params, group))
    df_group.append(exp.process_byGroup_ROM(df_brazo,df_params, group))

data = {
    'grupo': group_list,
    'dataframe': df_group
}

df = pd.DataFrame(data)
df.sample(3)






,grupo,dataframe
1,moderado,user ROM_Q1 ...
0,leve,user ROM_...
2,sano,user ROM_Q1 ROM_Q2...


In [ ]:
df.dataframe[2]

# Gráfico DTW

In [ ]:
# Plot DTW
index = 195
df = df_data.dataFrameIA[index]
tarea = df_data.tipo_actividad[index]
Sivarita.plotDTWparam(df, tarea)


In [ ]:
mean_org_dtw = []
mean_pred_dtw = []
user_list = []
df_DTW_list = []
tarea_pred = []
idAct = []

for index, rowData in df_data.iterrows():
    idAct.append(rowData['tipo_actividad'])
    user_list.append(rowData['usuario'])
    for subIndex, rowIA in rowData['dataFrameIA'].iterrows():
        # Calcular las medias de todas las joints
        sum_org = rowIA['data_org_1'] + rowIA['data_org_2'] + rowIA['data_org_3'] + rowIA['data_org_4'] + rowIA['data_org_5'] + rowIA['data_org_6'] + rowIA['data_org_7']
        sum_pred = rowIA['data_pred_1'] + rowIA['data_pred_2'] + rowIA['data_pred_3'] + rowIA['data_pred_4'] + rowIA['data_pred_5'] + rowIA['data_pred_6'] + rowIA['data_pred_7']
        mean_org = sum_org / 7
        mean_pred = sum_pred / 7

        mean_org_dtw.append(mean_org)
        mean_pred_dtw.append(mean_pred)
        tarea_pred.append(Sivarita.getActivityType2(rowIA['tarea']))

    DTW_mean = {
        'mean_org_dtw': mean_org_dtw,
        'mean_pred_dtw': mean_pred_dtw,
        'tarea_pred': tarea_pred
    }
    df_DTW_mean = pd.DataFrame(DTW_mean)
    df_DTW_list.append(df_DTW_mean)

    mean_org_dtw.clear()
    mean_pred_dtw.clear()
    tarea_pred.clear()
    


DTW = {
    'usuario': user_list,
    'tipo_actividad': idAct,
    'df_DTW_mean': df_DTW_list
}

df_DTW = pd.DataFrame(DTW)



In [ ]:
def processByGroupDTW(df_main, df_DTW, grp):
    group_DTW_list = []
    nombres = df_main[df_main.grupo == grp].nombre.to_list()
    for user in nombres:
        for i, rowDTW in df_DTW.iterrows():
            if(user == rowDTW['usuario']):
                group_DTW_list.append(rowDTW)
    
    group_DTW = pd.DataFrame(group_DTW_list)

    return group_DTW


df_subGroupDTW = []
group_list2 = []
for group in hawer:
    # filtrar grupo
    group_list2.append(group)
    df_subGroupDTW.append(processByGroupDTW(df_brazo, df_DTW, group))


dataDTW = {
'grupo': group_list2,
'dataframeDTW': df_subGroupDTW
}

df_groupDTW = pd.DataFrame(dataDTW)

In [ ]:
aux_df = df_groupDTW.dataframeDTW[2]

Plot DTW

In [ ]:

x_values  = []
y_values = []
for index1, row_group in df_groupDTW.iterrows():
    for index2, sub_row_group in row_group['dataframeDTW'].iterrows():
        if(row_group['grupo'] == 'sano'):
            color_marker = 'red'
        elif(row_group['grupo'] == 'leve'):
            color_marker = 'green'
        elif(row_group['grupo'] == 'moderado'):
            color_marker = 'blue'
        
        for index3, sub_row_DTW in sub_row_group['df_DTW_mean'].iterrows():
            # x_values.append(index2)
            # y_values.append(sub_row_DTW['mean_org_dtw'])
            if((sub_row_DTW['mean_org_dtw'] < 40) or (sub_row_DTW['mean_pred_dtw'] < 40)):
                if( sub_row_DTW['tarea_pred']== sub_row_group['tipo_actividad']):
                    plt.scatter(sub_row_DTW['mean_org_dtw'], sub_row_DTW['mean_pred_dtw'], c=color_marker, marker='o')
                elif( sub_row_DTW['tarea_pred']!= sub_row_group['tipo_actividad']):
                    plt.scatter(sub_row_DTW['mean_org_dtw'], sub_row_DTW['mean_pred_dtw'], c=color_marker, marker='x')


# fig = px.scatter(x=x_values, y=y_values)
# fig.show()
#plt.scatter(x, y)
plt.xlabel('DTW distance to True Gesture')
plt.ylabel('DTW distance to Predicted Gesture')
plt.title('Gráfico de DTW ')
plt.legend()
plt.show()

# Gráficas

Boxplots por Q

In [19]:
q_data = []
q_groups = []

for i_q in range(1,8):

    qg = []
    ql = []
        
    for i_group in [1, 0, 2]:
        
        df_tmp = df.dataframe[i_group]
        
        qg.append(df.grupo[i_group])
        #ql.append( ((df_tmp[str("mean_MAXQ"+str(i_q))]) - (df_tmp[str("mean_MINQ"+str(i_q))])).to_list())
        ql.append( (df_tmp[str("ROM_Q"+str(i_q))]).to_list())

    q_groups.append(qg)
    q_data.append(ql)

row = [1,1,1,1,2,2,2]
col = [1,2,3,4,1,2,3]

fig = make_subplots(rows=2, cols=4, subplot_titles=("Q1","Q2","Q3","Q4","Q5","Q6","Q7"))
for i_plot in range(7):
    fig.add_trace(go.Box(y=np.array(q_data[i_plot][1]),marker_color=px.colors.qualitative.T10[1],name=q_groups[i_plot][1]),row=row[i_plot],col=col[i_plot])
    fig.add_trace(go.Box(y=np.array(q_data[i_plot][0]),marker_color=px.colors.qualitative.T10[2],name=q_groups[i_plot][0]),row=row[i_plot],col=col[i_plot])
    fig.add_trace(go.Box(y=np.array(q_data[i_plot][2]),marker_color=px.colors.qualitative.T10[3],name=q_groups[i_plot][2]),row=row[i_plot],col=col[i_plot])

fig.update_layout(
    yaxis1 = dict(range=[0, 250]),
    yaxis2 = dict(range=[0, 250]),
    yaxis3 = dict(range=[0, 250]),
    yaxis4 = dict(range=[0, 250]),
    yaxis5 = dict(range=[0, 250]),
    yaxis6 = dict(range=[0, 250]),
    yaxis7 = dict(range=[0, 250]),
    showlegend = False
    )

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show() 

In [ ]:
q_data = []
q_groups = []

for i_q in range(1,8):

    qg = []
    ql = []
        
    for i_group in [1, 0, 2]:
        
        df_tmp = df.dataframe[i_group]
        
        qg.append(df.grupo[i_group])
        ql.append( (df_tmp[str("mean_MAXQ"+str(i_q))] - df_tmp[str("mean_MINQ"+str(i_q))]).to_list() )

    q_groups.append(qg)
    q_data.append(ql)

row = [1,1,1,1,2,2,2]
col = [1,2,3,4,1,2,3]

fig = make_subplots(rows=1, cols=1)
for i_plot in range(7):
    
    fig.add_trace(go.Bar(name = q_groups[i_plot][1],
        x=[1+i_plot+0.19],
        y=[np.mean(q_data[i_plot][1])],
        text=[np.round(np.mean(q_data[i_plot][1]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[1],       
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(q_data[i_plot][1])]),
        )        
    )
    
    fig.add_trace(go.Bar(name = q_groups[i_plot][0],
        x=[1+i_plot-0.19],
        y=[np.mean(q_data[i_plot][0])],
        text=[np.round(np.mean(q_data[i_plot][0]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[0],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(q_data[i_plot][0])]),
        )
    )

    fig.add_trace(go.Bar(name = q_groups[i_plot][2],
        x=[1+i_plot-0.19],
        y=[np.mean(q_data[i_plot][2])],
        text=[np.round(np.mean(q_data[i_plot][2]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[2],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(q_data[i_plot][2])]),
        )
    )
    
    

fig.update_layout(
    yaxis = dict(range=[0, 80]),
    showlegend = True
    )

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show() 

DTW Groups

In [20]:

offset = 25
max_size = 18

fig = go.Figure()

#Diagonal
fig.add_trace(
            go.Scatter(
                mode='lines',
                x=[0, max_size],
                y=[0, max_size],
                showlegend=False,
                marker=dict(
                    color='gray',
                )
            )
        )

#Plot Datos
for index, row in df.iterrows():
    if(row['grupo'] == 'sano'):
        color_marker = 'red'
    elif(row['grupo'] == 'leve'):
        color_marker = 'green'
    elif(row['grupo'] == 'moderado'):
        color_marker = 'blue'
    fig.add_trace(
            go.Scatter(
                mode='markers',
                x=row.dataframe['mean_dtw_org_error'],
                y=row.dataframe['mean_dtw_error'],
                text=row.dataframe['user'],
                name=row['grupo'],
                marker=dict(
                    color=color_marker,
                )
            )
        )

#Update Layaout
fig.update_layout(
    title="Gráfico DTW",
    xaxis_title="mean DTW distance to True Gesture",
    yaxis_title="mean DTW distance to Predicted Gesture",
    legend_title="Grupos"
    )

fig.show()

Predicción

In [22]:
group = []
dtw = []
pred = []    
for i_group in [1, 0, 2]:    
    df_tmp = df.dataframe[i_group]
    
    group.append(df.grupo[i_group])    
    dtw.append( df_tmp.mean_dtw_error.to_list() )
    pred.append( df_tmp.mean_porc_pred.to_list() )

fig = make_subplots(rows=1, cols=2, subplot_titles=("DTW", "% Predicción"))
for i_plot in range(3):
    
    fig.add_trace(go.Bar(name = group[i_plot][i_plot],
        x=[1+i_plot-0.19],
        y=[np.mean(dtw[i_plot][i_plot])],
        text=[np.round(np.mean(dtw[i_plot]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[i_plot],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(dtw[i_plot])])
        ),col=1,row=1
    )
    
    fig.add_trace(go.Bar(name = group[i_plot][i_plot],
        x=[1+i_plot+0.19],
        y=[np.mean(pred[i_plot][i_plot])],
        text=[np.round(np.mean(pred[i_plot]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[i_plot],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(pred[i_plot])])
        ),col=2,row=1
    )

fig.update_layout(
yaxis2 = dict(range=[0, 100]),
showlegend = True
)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show() 